## Numerical simulations of ideal chain model of polymer using PYTHON

### PYTHON LIBRARIES

In [2]:
import numpy as np
import math

### FJC PARAMETERS

In [3]:
n= [10,50,100,250,500,750,1000]
b=3.0
T=5000

### SIMULATION

In [4]:
for N in n:
    
    # OPEN TRAJECTORY FILE
    filename='trajectory_polymer_b=%.1f_N=%d_T=%d.xyz'%(b,N,T)
    f=open(filename,'w')

    #OPEN Q TIME SERIES FILE
    filename='end-to-end_distance_b=%.1f_N=%d_T=%d.dat'%(b,N,T)
    f2 = open(filename,'w')

    #OPEN Rg TIME SERIES FILE
    filename='gyration_radius_b=%.1f_N=%d_T=%d.dat'%(b,N,T)
    f3 = open(filename,'w')

    Q_T = np.zeros(T)
    R_g = np.zeros(T)

    for t in range(T):
        
        # INITIALIZE
        vR=np.zeros([N+1,3])
        vb=np.zeros([N,3])

        # MONOMER 0
        i=0
        vR[i,:]=np.array([0.0,0.0,0.0])

        # MONOMER 1
        i=1
        vb[i-1,:]=np.array([b,0.0,0.0])
        vR[i,:]=vR[i-1,:]+vb[i-1,:]

        # MONOMER 2
        i=2
        u=np.random.uniform(-1.0,1.0)
        tau=math.atan2(math.sqrt(1-u*u),u)
        vN=np.array([0.0,0.0,1.0]); vN=vN/np.linalg.norm(vN)
        vU=vb[0,:]
        vV=math.cos(math.pi-tau)*vU+math.sin(math.pi-tau)*np.cross(vN,vU)+(1-math.cos(math.pi-tau))*np.dot(vU,vN)*vN
        vb[i-1,:]=vV
        vR[i,:]=vR[i-1,:]+vb[i-1,:]

        # MONOMER > 2
        for i in range(3,N+1):
            u=np.random.uniform(-1.0,1.0)
            tau=math.atan2(math.sqrt(1-u*u),u)
            vN=np.cross(vb[i-3,:],vb[i-2,:]); vN=vN/np.linalg.norm(vN)
            vU=vb[i-2,:]
            vV=math.cos(math.pi-tau)*vU+math.sin(math.pi-tau)*np.cross(vN,vU)+(1-math.cos(math.pi-tau))*np.dot(vU,vN)*vN

            u=np.random.uniform(-1.0,1.0); v=np.random.uniform(-1.0,1.0)
            phi=math.atan2(v,u)
            vN=vb[i-2,:]; vN=vN/np.linalg.norm(vN)
            vU=vV
            vV=math.cos(phi)*vU+math.sin(phi)*np.cross(vN,vU)+(1-math.cos(phi))*np.dot(vU,vN)*vN
            vb[i-1,:]=vV
            vR[i,:]=vR[i-1,:]+vb[i-1,:]
    
        # CENTER OF MASS
        Rcm=np.mean(vR,axis=0)
        vR=vR-Rcm
    
        Q_T[t] = np.linalg.norm(vR[-1,:] - vR[0,:])
        R_g[t] = math.sqrt(np.mean(np.linalg.norm(vR,axis=1)**2))
   
        # OUTPUT XYZ TRAJECTORY     
        f.write('%d\n'%(N+1))
        f.write('T=%d\n'%t)
        for n in range(N+1):
            f.write('%s %f %f %f\n'%('C',vR[n,0],vR[n,1],vR[n,2]))
        #OUTPUT Q TIME SERIES 
        f2.write('%d %f\n'%(t,Q_T[t]))
    
        #OUTPUT R_g TIME SERIES
        f3.write('%d %f\n'%(t,R_g[t]))
    # CLOSE TRAJECTORY FILE
    f.close()
    f2.close()
    f3.close()

### METRICS

\begin{equation}
    Q=\left\lVert \vec{R}_N-\vec{R}_0 \right\rVert
\end{equation}

\begin{equation}
    R_g=\sqrt{\dfrac{1}{N+1}\sum_{i=0}^{N}\left\lVert\vec{R}_i-\vec{R}_{cm}\right\lVert^2}
\end{equation}